In [1]:
import numpy as np
import pandas as pd
import torch

def find_projection(y, c):
    y_hat = np.concatenate((y, y - 1), axis=0)
    y_hat = np.sort(y_hat)
    list_S = []
    for k in range(len(y_hat)):
        S_k = 0
        for l in range(len(y)):
            S_k += max(0, min(1, y[l] - y_hat[k]))
        list_S.append(S_k)
    max_k = 0
    for k in range(len(list_S) - 1):
        if list_S[k] >= c and max_k < k:
            max_k = k
    ld = y_hat[max_k] + ((c-list_S[max_k])/(list_S[max_k+1]-list_S[max_k])) * (y_hat[max_k+1]-y_hat[max_k])
    list_x_bar_k = []
    for k in range(len(y)):
        x_bar_k = max(0, min(1, y[k] - ld))
        list_x_bar_k.append(x_bar_k)
    return list_x_bar_k

def func_F(y, z, X):
    out = 0
    for i in range(X.shape[0]):
        for k in range(X.shape[1]):
            out += (X[i][k] - y[i]*z[k])**2
    return (1/2) * out

def grad_F(y, z, X):
    y_clone = y.clone()
    y_clone = y_clone.detach().requires_grad_()
    out = func_F(y_clone, z, X)
    out.backward()
    return y_clone.grad

def project_grad_descent(X, z, eta, alpha, T):
    for i in range(T):
        eta = torch.tensor(find_projection((eta-alpha*grad_F(eta, z, X)).numpy(), 1), dtype = torch.float64)
    return eta

In [2]:
if __name__ == "__main__":
    X = pd.read_csv('homework3_data/project3_matrix.csv')
    X = X.drop(['Unnamed: 0'], axis=1).to_numpy()
    X = torch.tensor(X, dtype = torch.float64)
    alpha, T = 1e-5, 50
    eta = torch.tensor([1/X.shape[0]] * X.shape[0], dtype = torch.float64)
    z = (X.shape[0])**2 * torch.matmul(X.t(), eta)
    eta = project_grad_descent(X, z, eta, alpha, T).numpy() 
    list_k, list_eta_k = [], []
    for k in range(len(eta)):
        if eta[k] > 0.001:
            list_k.append(k + 1)
            list_eta_k.append(eta[k])
    print("List of indexes k for which " + r'$\eta_T^{(k)}>0.001$' + " is: " + str(list_k))
    print("The corresponding values of " + r'$\eta_T^{(k)}$' + " is: " + str(list_eta_k))

List of indexes k for which $\eta_T^{(k)}>0.001$ is: [24, 36, 63, 96]
The corresponding values of $\eta_T^{(k)}$ is: [0.2802846365138505, 0.4170689676071295, 0.18152550727653338, 0.1211208886024866]


In [3]:
def get_optim_z(y, X):
    list_z = []
    for k in range(X.shape[1]):
        sum_Xy = 0
        for i in range(X.shape[0]):
            sum_Xy += X[i][k] * y[i]
        sum_Xy = sum_Xy/(torch.norm(y)**2)
        list_z.append(max(0, sum_Xy))
    return list_z

def project_grad_descent_y(X, y, alpha, T):
    for i in range(100):
        z_y = torch.tensor(get_optim_z(y, X), dtype = torch.float64)
        y = project_grad_descent(X, z_y, y, alpha, T)
    return y

In [4]:
if __name__ == "__main__":
    alpha, T = 1e-5, 10
    y = torch.tensor([1/X.shape[0]] * X.shape[0], dtype = torch.float64)
    y = project_grad_descent_y(X, y, alpha, T).numpy()
    list_k, list_y_k = [], []
    for k in range(len(y)):
        if y[k] > 0.001:
            list_k.append(k + 1)
            list_y_k.append(y[k])
    print("List of indexes k for which " + r'$y^{(k)}>0.001$' + " is: " + str(list_k))
    print("The corresponding values of " + r'$y^{(k)}$' + " is: " + str(list_y_k))

List of indexes k for which $y^{(k)}>0.001$ is: [18, 24, 36, 46, 52, 58, 63, 72, 80, 96]
The corresponding values of $y^{(k)}$ is: [0.109082660499241, 0.0962864473347163, 0.10164719426063612, 0.08132085817922366, 0.11305297011170733, 0.11095008460546285, 0.09241543700155856, 0.11335755192270829, 0.07842293120707078, 0.09005381861309705]
